<a href="https://colab.research.google.com/github/DoraSolares/PysparkColaboratory/blob/master/PySpark_Dora_Solares.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install pyspark in colab

---

To run spark in Colab, first we need to install all the dependencies in Colab environment such as Apache Spark 2.4.7 with hadoop 2.7, Java 8 and Findspark in order to locate the spark in the system. The tools installation can be carried out inside the Jupyter Notebook of the Colab.

In [15]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

Now that we have installed Spark and Java in Colab, it is time to set the environment path that enables us to run PySpark in our Colab environment. Set the location of Java and Spark by running the following code:

In [16]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

We can run a local spark session to test our installation:

In [17]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

Our Colab is ready to run PySpark.

## Dataset
---
### Context
The dataset was originally collected by Austin Reese on 2020-01-07. The source of this dataset is Craiglist.org. The domain of the dataset is Public.
### Content
The dataset comprises of 265190 house records with 22 columns. You can find more details in the housePricing.docx file attached with the dataset.

In [18]:
!wget https://github.com/DoraSolares/PysparkColaboratory/raw/master/housing_train.csv.zip
!unzip housing_train.csv.zip

--2021-05-18 01:32:27--  https://github.com/DoraSolares/PysparkColaboratory/raw/master/housing_train.csv.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/DoraSolares/PysparkColaboratory/master/housing_train.csv.zip [following]
--2021-05-18 01:32:27--  https://media.githubusercontent.com/media/DoraSolares/PysparkColaboratory/master/housing_train.csv.zip
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76443248 (73M) [application/zip]
Saving to: ‘housing_train.csv.zip.1’

housing_train.csv.z 100%[===================>]  72.90M  96.6MB/s    in 0.8s    

2021-05-18 01:32:28 (96.6 MB

In [19]:
df = spark.read.csv('/content/housing_train.csv',inferSchema=True, header =True)
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- url: string (nullable = true)
 |-- region: string (nullable = true)
 |-- region_url: string (nullable = true)
 |-- price: string (nullable = true)
 |-- type: string (nullable = true)
 |-- sqfeet: string (nullable = true)
 |-- beds: string (nullable = true)
 |-- baths: string (nullable = true)
 |-- cats_allowed: string (nullable = true)
 |-- dogs_allowed: string (nullable = true)
 |-- smoking_allowed: string (nullable = true)
 |-- wheelchair_access: string (nullable = true)
 |-- electric_vehicle_charge: integer (nullable = true)
 |-- comes_furnished: string (nullable = true)
 |-- laundry_options: string (nullable = true)
 |-- parking_options: string (nullable = true)
 |-- image_url: string (nullable = true)
 |-- description: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- long: string (nullable = true)
 |-- state: string (nullable = true)



In [20]:
df.count()

265193

In [21]:
df.select("price").distinct().show()

+-----+
|price|
+-----+
|  675|
| 1090|
|  829|
| 1159|
| 1572|
|  467|
|  691|
| 1512|
| 1436|
| 2162|
| 3210|
| 2069|
| 2088|
| 2136|
| 4032|
|  296|
|  800|
|  853|
| 2275|
| 1372|
+-----+
only showing top 20 rows



In [22]:
df.select("type").distinct().show()

+---------------+
|           type|
+---------------+
| Community Pool|
|         in-law|
|           loft|
|           null|
|         duplex|
|      townhouse|
|           flat|
|          condo|
|            Gym|
|   manufactured|
|           land|
|          house|
|assisted living|
|      apartment|
|  cottage/cabin|
+---------------+



In [23]:
df.select(["id","region"]).filter(df.price<1000).show()

+----------+----------+
|        id|    region|
+----------+----------+
|7041966914|birmingham|
|7041966936|birmingham|
|7041966888|birmingham|
|7041966868|birmingham|
|7041966702|birmingham|
|7041959413|birmingham|
|7041958307|birmingham|
|7041938996|birmingham|
|7041935244|birmingham|
|7041934777|birmingham|
|7037827287|birmingham|
|7040738380|birmingham|
|7041943185|birmingham|
|7041943167|birmingham|
|7041934856|birmingham|
|7041926275|birmingham|
|7041926645|birmingham|
|7041926412|birmingham|
|7040738422|birmingham|
|7041915578|birmingham|
+----------+----------+
only showing top 20 rows



In [24]:
apartments=df.select(["id","price", "region", "sqfeet", "beds", "baths", "parking_options"]).filter("type='apartment' AND price<1000")
apartments.show()

+----------+-----+----------+------+----+-----+------------------+
|        id|price|    region|sqfeet|beds|baths|   parking_options|
+----------+-----+----------+------+----+-----+------------------+
|7041966914|  825|birmingham|  1133|   1|  1.5|    street parking|
|7041966936|  800|birmingham|   927|   1|    1|    street parking|
|7041966888|  785|birmingham|  1047|   2|    1|    street parking|
|7041966868|  900|birmingham|  1298|   2|    2|    street parking|
|7041966702|  925|birmingham|  1350|   2|    2|    street parking|
|7041959413|  799|birmingham|   703|   1|    1|              null|
|7041958307|  890|birmingham|  1050|   2|    2|              null|
|7041935244|  870|birmingham|   851|   1|    1|off-street parking|
|7041934777|  815|birmingham|   851|   1|    1|off-street parking|
|7037827287|  700|birmingham|   906|   1|    1|              null|
|7040738380|  820|birmingham|   851|   1|    1|off-street parking|
|7041943185|  925|birmingham|  1350|   2|    2|    street park

In [25]:
apartments.count()

100261

In [26]:
ap_with_parking = apartments.filter("parking_options='off-street parking' AND region<>'birmingham'")
ap_with_parking.show()

+----------+-----+--------------------+------+----+-----+------------------+
|        id|price|              region|sqfeet|beds|baths|   parking_options|
+----------+-----+--------------------+------+----+-----+------------------+
|7045091151|  855|huntsville / decatur|   856|   1|    1|off-street parking|
|7045924966|  700|              dothan|   950|   2|    1|off-street parking|
|7032696630|  710|              dothan|   641|   1|    1|off-street parking|
|7032002167|  710|              dothan|   641|   1|    1|off-street parking|
|7032688948|  710|              dothan|   641|   1|    1|off-street parking|
|7032700160|  710|              dothan|   641|   1|    1|off-street parking|
|7046773373|  710|              dothan|   641|   1|    1|off-street parking|
|7027420859|  710|              dothan|   641|   1|    1|off-street parking|
|7027414691|  710|              dothan|   641|   1|    1|off-street parking|
|7027429517|  710|              dothan|   641|   1|    1|off-street parking|

In [27]:
ap_with_parking.count()

44489

In [28]:
ap_with_parking.write.mode("overwrite").csv("/content/cheap_apartments")